In [1]:
!nvidia-smi

Thu Apr 11 10:36:39 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.103.01   Driver Version: 470.103.01   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:05:00.0 Off |                    0 |
| N/A   32C    P8    26W / 149W |     29MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla K80           Off  | 00000000:06:00.0 Off |                    0 |
| N/A   

In [2]:
import os
import nibabel as nib
import numpy as np 
from collections import OrderedDict
import json
from pathlib import Path

from utils.helper import convert_nrrd_to_nifti, create_folder, plot_all_slices, plot_histogram, generate_binary_image, adjust_affine_for_spacing_and_origin, save_binary_image_with_adjusted_origin, make_if_dont_exist
from utils.metrics import dice_score_per_class, hausdorff_distance_per_class, ravd_per_class

In [3]:
# define dataset path
BASE_PATH = Path('./').resolve()
DATA_PATH = BASE_PATH / 'dataset'

project_name = 'HPC' #change here for different task name
task_name = 'Dataset003_' + project_name 

TRAINING_DATASET_PATH = BASE_PATH / 'dataset/nnUNet_raw_data' / task_name / 'imagesTr'
GT_TRAINING_DATASET_PATH = BASE_PATH / 'dataset/nnUNet_raw_data' / task_name / 'labelsTr'
TEST_DATASET_PATH = BASE_PATH / 'dataset/nnUNet_raw_data' / task_name / 'imagesTs'
GT_TEST_DATASET_PATH = BASE_PATH / 'dataset/nnUNet_raw_data' / task_name / 'labelsTs'
PREDICTION_RESULTS_PATH  = BASE_PATH / 'dataset/nnUNet_Prediction_Results' / task_name
TASK_PATH = BASE_PATH / 'dataset/nnUNet_raw_data' / task_name 

# setup environment variables
nnUNet_raw = BASE_PATH / 'dataset/nnUNet_raw_data'
nnUNet_preprocessed = BASE_PATH / 'dataset/nnUNet_preprocessed'
nnUNet_results = BASE_PATH / 'dataset/nnUNet_results'

In [12]:
train_files = os.listdir(TRAINING_DATASET_PATH)
label_files = os.listdir(GT_TRAINING_DATASET_PATH)
print("train image files:",len(train_files))
print("train label files:",len(label_files))
print("Matches:",len(set(train_files).intersection(set(label_files))))


train image files: 9
train label files: 9
Matches: 0


In [4]:
%env nnUNet_raw=$nnUNet_raw
%env nnUNet_preprocessed=$nnUNet_preprocessed
%env nnUNet_results=$nnUNet_results

env: nnUNet_raw=/beegfs/data/work/shared/ngmm/scripts/Taiabur/ngmm-nnunet/dataset/nnUNet_raw_data
env: nnUNet_preprocessed=/beegfs/data/work/shared/ngmm/scripts/Taiabur/ngmm-nnunet/dataset/nnUNet_preprocessed
env: nnUNet_results=/beegfs/data/work/shared/ngmm/scripts/Taiabur/ngmm-nnunet/dataset/nnUNet_results


In [22]:
!nnUNetv2_plan_and_preprocess -d 3 --verify_dataset_integrity

Fingerprint extraction...
Dataset003_HPC
Using <class 'nnunetv2.imageio.simpleitk_reader_writer.SimpleITKIO'> as reader/writer

####################
verify_dataset_integrity Done. 
If you didn't see any error messages then your dataset is most likely OK!
####################

Experiment planning...
Attempting to find 3d_lowres config. 
Current spacing: [1.03 1.03 1.03]. 
Current patch size: (112, 224, 96). 
Current median shape: [401.94174757 818.44660194 341.74757282]
Attempting to find 3d_lowres config. 
Current spacing: [1.0609 1.0609 1.0609]. 
Current patch size: (112, 224, 96). 
Current median shape: [390.23470638 794.6083514  331.79376002]
Attempting to find 3d_lowres config. 
Current spacing: [1.092727 1.092727 1.092727]. 
Current patch size: (112, 224, 96). 
Current median shape: [378.86864697 771.46441883 322.12986409]
Attempting to find 3d_lowres config. 
Current spacing: [1.12550881 1.12550881 1.12550881]. 
Current patch size: (112, 224, 96). 
Current median shape: [367.8336

In [ ]:
!nnUNetv2_train 3 3d_lowres 0 --npz 

In [ ]:
!nnUNetv2_train 3 3d_lowres 1 --npz 

In [ ]:
!nnUNetv2_train 3 3d_lowres 2 --npz 

In [ ]:
!nnUNetv2_train 3 3d_lowres 3 --npz 

In [ ]:
!nnUNetv2_train 3 3d_lowres 4 --npz 

In [5]:
!nnUNetv2_find_best_configuration 3 -c 3d_lowres 



***All results:***
nnUNetTrainer__nnUNetPlans__3d_lowres: 0.8974329107317696

*Best*: nnUNetTrainer__nnUNetPlans__3d_lowres: 0.8974329107317696

***Determining postprocessing for best model/ensemble***
Results were improved by removing all but the largest foreground region. Mean dice before: 0.89743 after: 0.89762

***Run inference like this:***

nnUNetv2_predict -d Dataset003_HPC -i INPUT_FOLDER -o OUTPUT_FOLDER -f  0 1 2 3 4 -tr nnUNetTrainer -c 3d_lowres -p nnUNetPlans

***Once inference is completed, run postprocessing like this:***

nnUNetv2_apply_postprocessing -i OUTPUT_FOLDER -o OUTPUT_FOLDER_PP -pp_pkl_file /beegfs/data/work/shared/ngmm/scripts/Taiabur/ngmm-nnunet/dataset/nnUNet_results/Dataset003_HPC/nnUNetTrainer__nnUNetPlans__3d_lowres/crossval_results_folds_0_1_2_3_4/postprocessing.pkl -np 8 -plans_json /beegfs/data/work/shared/ngmm/scripts/Taiabur/ngmm-nnunet/dataset/nnUNet_results/Dataset003_HPC/nnUNetTrainer__nnUNetPlans__3d_lowres/crossval_results_folds_0_1_2_3_4/plan

### prediction

In [6]:
!nnUNetv2_predict -d Dataset003_HPC -i /dataset/nnUNet_test_data -o /dataset/nnUNet_Prediction_Results/Dataset003_HPC -f  0 1 2 3 4 -tr nnUNetTrainer -c 3d_lowres -p nnUNetPlans


#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

There are 1 cases in the source folder
I am process 0 out of 1 (max process ID is 0, we start counting with 0!)
There are 1 cases that I would like to predict

Predicting NG4127_RCL5:
perform_everything_on_device: True
100%|███████████████████████████████████████████| 18/18 [03:09<00:00, 10.53s/it]
sending off prediction to background worker for resampling and export
done with NG4127_RCL5


### post processing

In [7]:
!nnUNetv2_apply_postprocessing -i /user1/ngmm/tr855969/Desktop/Taiabur/ngmm-nnunet/dataset/nnUNet_Prediction_Results/Dataset003_HPC -o /user1/ngmm/tr855969/Desktop/Taiabur/ngmm-nnunet/dataset/nnUNet_postprocessed/Dataset003_HPC/3d_lowers -pp_pkl_file /beegfs/data/work/shared/ngmm/scripts/Taiabur/ngmm-nnunet/dataset/nnUNet_results/Dataset003_HPC/nnUNetTrainer__nnUNetPlans__3d_lowres/crossval_results_folds_0_1_2_3_4/postprocessing.pkl -np 8 -plans_json /beegfs/data/work/shared/ngmm/scripts/Taiabur/ngmm-nnunet/dataset/nnUNet_results/Dataset003_HPC/nnUNetTrainer__nnUNetPlans__3d_lowres/crossval_results_folds_0_1_2_3_4/plans.json

In [4]:
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

# Load the NIfTI file
nifti_path = '/user1/ngmm/tr855969/Desktop/Taiabur/ngmm-nnunet/dataset/nnUNet_raw_data/Dataset002_HCFC1/imagesTr/NG4108_RCL5_0000.nii.gz'
# org_nifti_path = '/user1/ngmm/tr855969/Desktop/Taiabur/ngmm-nnunet/dataset/nnUNet_raw_data/Dataset003_HPC/imagesTr/NG4115_RCL5_0000.nii.gz'
img = nib.load(nifti_path)
data = img.get_fdata()

# org_img = nib.load(org_nifti_path).get_fdata()

# data = org_img * data
# Adjusted function to display a slice and its histogram
def display_slice_and_histogram(slice_no):
    # Setup the figure and axes for a side-by-side plot: slice and histogram
    fig, axes = plt.subplots(2, 2, figsize=(16, 8))
    
    # Display the slice
    ax = axes[0,0]
    ax.imshow(data[:, :, slice_no])
    ax.axis('off')  # Hide axes ticks
    ax.set_title(f'Slice {slice_no}')
    
    # Display the histogram
    ax = axes[0,1]
    slice_data = data[:, :, slice_no].ravel()
    ax.hist(slice_data, bins=50, color='c', alpha=0.75)
    ax.set_title('Pixel Intensity Distribution')
    ax.grid(True)

    ax = axes[1,0]
    ax.imshow(data[:, slice_no,: ].T)
    ax.axis('off')  # Hide axes ticks
    ax.set_title(f'Slice {slice_no}')

    ax = axes[1,1]
    ax.imshow(data[slice_no, :,: ])
    ax.axis('off')  # Hide axes ticks
    ax.set_title(f'Slice {slice_no}')
    
    plt.tight_layout()
    plt.show()

# Interactive widget for slice selection
slice_slider = widgets.IntSlider(
    min=0, 
    max=data.shape[2] - 1, 
    step=1, 
    value=data.shape[2] // 2, 
    description='Slice'
)

widgets.interactive(display_slice_and_histogram, slice_no=slice_slider)


interactive(children=(IntSlider(value=201, description='Slice', max=401), Output()), _dom_classes=('widget-int…

In [10]:
import os
import shutil

def copy_files(source_dir, destination_dir):
    # Create the destination directory if it doesn't exist
    if not os.path.exists(destination_dir):
        os.makedirs(destination_dir)

    # Get a list of all files in the source directory
    files = os.listdir(source_dir)

    # Copy each file to the destination directory
    for file in files:
        source_file = os.path.join(source_dir, file)
        destination_file = os.path.join(destination_dir, file)
        shutil.copy2(source_file, destination_file)
        print(f"Copied '{source_file}' to '{destination_file}'")

# Example usage
source_directory = '/user1/ngmm/tr855969/Desktop/Taiabur/ngmm-nnunet/dataset/nnUNet_results/Dataset002_HCFC1/nnUNetTrainer__nnUNetPlans__3d_lowres/fold_1/validation/'
destination_directory = '/work/shared/ngmm/3Dimage/DL_test/validation/'

copy_files(source_directory, destination_directory)


Copied '/user1/ngmm/tr855969/Desktop/Taiabur/ngmm-nnunet/dataset/nnUNet_results/Dataset002_HCFC1/nnUNetTrainer__nnUNetPlans__3d_lowres/fold_1/validation/NG4117_RCL5.npz' to '/work/shared/ngmm/3Dimage/DL_test/validation/NG4117_RCL5.npz'


KeyboardInterrupt: 

In [3]:
source_file = '/user1/ngmm/tr855969/Desktop/Taiabur/ngmm-nnunet/dataset/nnUNet_results/Dataset002_HCFC1/nnUNetTrainer__nnUNetPlans__3d_lowres/fold_1/validation/NG4115_RCL5.nii.gz'
destination_file = '/work/shared/ngmm/3Dimage/DL_test/validation/NG4115_RCL5.nii.gz'
import os
import shutil
shutil.copy2(source_file, destination_file)
print(f"Copied '{source_file}' to '{destination_file}'")


Copied '/user1/ngmm/tr855969/Desktop/Taiabur/ngmm-nnunet/dataset/nnUNet_results/Dataset002_HCFC1/nnUNetTrainer__nnUNetPlans__3d_lowres/fold_1/validation/NG4115_RCL5.nii.gz' to '/work/shared/ngmm/3Dimage/DL_test/validation/NG4115_RCL5.nii.gz'
